In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


df = pd.read_csv('UTKFace.csv')


df['pixels'] = df['pixels'].apply(lambda x: np.fromstring(x, dtype=int, sep=' ').reshape(48, 48, 1))

df['pixels'] = df['pixels'] / 255.0


X = np.stack(df['pixels'].values)
y_age = df['age'].values
y_gender = to_categorical(df['gender'].values)

# Split the data into training and testing sets
X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test = train_test_split(X, y_age, y_gender, test_size=0.3, random_state=101)


In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization


input_layer = Input(shape=(48, 48, 1))

# First block
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)

# Second block
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)


# Flattening and Dense layers
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

#gender output
gender_output = Dense(2, activation='softmax', name='gender_output')(x)  # For gender classification


In [4]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense

input_layer2 = Input(shape=(48, 48, 1))

# First block
x1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer2)
x1 = BatchNormalization()(x1)
x1 = Conv2D(32, (3, 3), activation='relu', padding='same')(x1)
x1 = BatchNormalization()(x1)
x1 = MaxPooling2D((2, 2))(x1)
x1 = Dropout(0.25)(x1)

# Second block
x1 = Conv2D(64, (3, 3), activation='relu', padding='same')(x1)
x1 = BatchNormalization()(x1)
x1 = Conv2D(64, (3, 3), activation='relu', padding='same')(x1)
x1 = BatchNormalization()(x1)
x1 = MaxPooling2D((2, 2))(x1)
x1 = Dropout(0.25)(x1)


# Flattening and Dense layers
x1 = Flatten()(x1)
x1 = Dense(512, activation='relu')(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.5)(x1)


# Output layer
age_output = Dense(1, activation='relu', name='age_output')(x1)  # For age prediction


In [5]:
model = Model(inputs=[input_layer, input_layer2], outputs=[age_output, gender_output])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 48, 48, 1)]          0         []                            
                                                                                                  
 conv2d_4 (Conv2D)           (None, 48, 48, 32)           320       ['input_1[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 48, 48, 32)           320       ['input_1[0][0]']             
                                                                                                  
 batch_normalization_5 (Bat  (None, 48, 48, 32)           128       ['conv2d_4[0][0]']            
 chNormalization)                                                                             

In [6]:
model.compile(optimizer='adam',
              loss={'age_output': 'mse', 'gender_output': 'binary_crossentropy'},
              metrics={'age_output': 'mse', 'gender_output': 'accuracy'})

model.fit([X_train,X_train], {'age_output': y_age_train, 'gender_output': y_gender_train},
          validation_data=([X_test,X_test], {'age_output': y_age_test, 'gender_output': y_gender_test}),
          epochs=50, batch_size=32)


Epoch 1/50
519/519 [==============================] - 266s 507ms/step - loss: 357.8376 - age_output_loss: 357.3891 - gender_output_loss: 0.4485 - age_output_mse: 357.3891 - gender_output_accuracy: 0.8141 - val_loss: 141.0404 - val_age_output_loss: 140.6146 - val_gender_output_loss: 0.4259 - val_age_output_mse: 140.6146 - val_gender_output_accuracy: 0.8145
Epoch 2/50
519/519 [==============================] - 259s 498ms/step - loss: 131.4013 - age_output_loss: 131.0816 - gender_output_loss: 0.3197 - age_output_mse: 131.0816 - gender_output_accuracy: 0.8625 - val_loss: 98.7117 - val_age_output_loss: 98.4153 - val_gender_output_loss: 0.2964 - val_age_output_mse: 98.4153 - val_gender_output_accuracy: 0.8722
Epoch 3/50
519/519 [==============================] - 260s 501ms/step - loss: 113.4884 - age_output_loss: 113.2071 - gender_output_loss: 0.2813 - age_output_mse: 113.2071 - gender_output_accuracy: 0.8800 - val_loss: 101.6103 - val_age_output_loss: 101.3200 - val_gender_output_loss: 0.29

In [7]:
model.save('gen_age_2.h5')


C:\Users\Hp\anaconda4\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.summary()
